In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
import os

project_path = "/content/drive/MyDrive/Dextro_Project"
os.chdir(project_path)
print("Working Directory:", os.getcwd())


Working Directory: /content/drive/MyDrive/Dextro_Project


In [4]:
!python train_model.py



2025-08-31 05:40:00.198657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756618800.224483    1985 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756618800.232966    1985 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756618800.254478    1985 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756618800.254518    1985 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756618800.254522    1985 computation_placer.cc:177] computation placer alr

In [5]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models


DATASET_DIR = "dataset"


IMG_SIZE = 128


CATEGORIES = ["Normal", "Dextrocardia"]


X = []
y = []

for label, category in enumerate(CATEGORIES):
    folder = os.path.join(DATASET_DIR, category)
    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)
        try:
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img)
            y.append(label)
        except Exception as e:
            print("Error loading image:", e)

X = np.array(X) / 255.0
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

print("Dataset Loaded!")
print("X shape:", X.shape)
print("y shape:", y.shape)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set:", X_train.shape)
print("Testing set:", X_test.shape)

np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)


Error loading image: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dataset Loaded!
X shape: (142, 128, 128, 1)
y shape: (142,)
Training set: (113, 128, 128, 1)
Testing set: (29, 128, 128, 1)


In [6]:
!ls



chest_xray	   dataset	    train_model.py  X_train.npy  y_train.npy
create_dataset.py  dextro_model.h5  X_test.npy	    y_test.npy


In [48]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score

# preprocessed data ko load krdiya
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

# labels for one hot encode
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

print("Data loaded for training!")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  # 2 classes
])


Data loaded for training!
X_train shape: (113, 128, 128, 1)
y_train shape: (113, 2)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
# Ttrain
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 749ms/step - accuracy: 0.9913 - loss: 0.0793

4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9913 - loss: 0.0792 - val_accuracy: 1.0000 - val_loss: 8.6358e-04
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 957ms/step - accuracy: 0.9913 - loss: 0.0583 - val_accuracy: 1.0000 - val_loss: 0.0107
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 773ms/step - accuracy: 0.9850 - loss: 0.0630 - val_accuracy: 1.0000 - val_loss: 0.0045
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 886ms/step - accuracy: 0.9913 - loss: 0.0672 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 966ms/step - accuracy: 0.9913 - loss: 0.0607 - val_accuracy: 1.0000 - val_loss: 0.0028
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 774ms/step - accuracy: 0.9850 - loss: 0.0863 - val_accuracy: 1.0000 - val_loss: 0.0062
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.9913 - loss: 0.0359 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 769ms/step - accuracy: 0.9913 - loss: 0.0241 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch

In [45]:

# Evaluate
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("Accuracy:", accuracy_score(y_true, y_pred_classes))
print(classification_report(y_true, y_pred_classes))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step
Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        29

    accuracy                           1.00        29
   macro avg       1.00      1.00      1.00        29
weighted avg       1.00      1.00      1.00        29



In [44]:

#model save
model.save("dextro_model.h5")


In [ ]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return "Flask is working on Colab with ngrok!"

app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [ ]:
!python app.py
